# Colab-specific instructions start here

Note to myself: a copy of this file should be in https://github.com/giorginolab/MD-Tutorial-Data/

In [2]:
# Here we use a Conda environment inside Google Colab. Blocks specific for Colab
# (like this one) mention "condacolab". On "normal" platforms the procedure
# for installation may be different - you need to check the system's documentation.

# Colab notebooks are "brittle": in the course of time Colab is updated
# and dependencies no longer work properly. Proper HPC platforms are more
# stable (and supported)

# After executing this cell, Colab restarts.

if 'google.colab' in str(get_ipython()):
  ! pip install -q condacolab
  import condacolab
  condacolab.install_miniforge()

✨🍰✨ Everything looks OK!


In [3]:
if 'google.colab' in str(get_ipython()):
  condacolab.check()

✨🍰✨ Everything looks OK!


In [4]:
# Colab-specific workaround for a weird error upon shell escape:
#   NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
if 'google.colab' in str(get_ipython()):
  import locale
  def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
  locale.getpreferredencoding = getpreferredencoding

# Install software

In [ ]:
# Install OpenMM. Takes a long time (unless already installed).
if 'google.colab' in str(get_ipython()):
  !conda install -q -c conda-forge openbabel smina rdkit mdtraj

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mdtraj
    - openbabel
    - rdkit
    - smina


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astunparse-1.6.3           |     pyhd8ed1ab_0          15 KB  conda-forge
    blosc-1.21.5               |       hc2324a3_1          48 KB  conda-forge
    brotli-1.1.0               |       hd590300_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hd590300_1          19 KB  conda-forge
    c-blosc2-2.14.4            |       hb4ffafa_1         329 KB  conda-forge
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    cairo-1.18.0               |       h3faef2a_0         959 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    chardet

In [ ]:
# Gnina is a GPU accelerated docking program, https://jcheminf.biomedcentral.com/articles/10.1186/s13321-021-00522-2
!wget -c https://github.com/gnina/gnina/releases/download/v1.1/gnina
!chmod a+x gnina
!mv gnina /usr/local/bin
!gnina --version

# Generic installation instructions

In [ ]:
# Verify Python version
import sys
print(sys.version)

# Verify GPU availability and type. If you get an error, check that
# "Runtime / Runtime type / GPU" is selected.
!nvidia-smi


# Begin

## SMILES strings encode (small) molecues

Note that they are not unique. ("Canonical SMILES" are, but they are still software-dependent).

In [ ]:
# https://www.rdkit.org/docs/GettingStartedInPython.html
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Lipinski import *

m = Chem.MolFromSmiles('Cc1ccccc1')
m

In [ ]:
from rdkit.Chem import Descriptors
import pandas as pd
descs = Descriptors.CalcMolDescriptors(m)
descs.keys()

In [ ]:
descs["NumRotatableBonds"]

In [ ]:
from rdkit.Chem import AllChem
AllChem.Compute2DCoords(m)
print(Chem.MolToMolBlock(m))

In [ ]:
mH = Chem.AddHs(m)
AllChem.EmbedMolecule(mH, AllChem.ETKDGv3())
print(Chem.MolToMolBlock(mH))


## Ibuprofen and aspirin

The SMILES string of aspirin is `CC(C)Cc1ccc(cc1)[C@H](C)C(=O)O`.

In [ ]:
ain_s="CC(=O)Oc1ccccc1C(=O)O"
ain=Chem.MolFromSmiles(ain_s)

In [ ]:
display(ain)


In [ ]:
ainH = Chem.AddHs(ain)
AllChem.EmbedMolecule(ainH, AllChem.ETKDGv3())
display(ainH)

In [ ]:
Descriptors.NumRotatableBonds(ain)

Ibuprofen has two enantiomers (one chiral center). (S)-ibuprofen is the active one. The (R) enantiomer is converted by an enzyme, so it becomes active as well. Here we specify the (R) stereochemistry from the get-go.

In [ ]:
ibp_s="CC(C)Cc1ccc(cc1)[C@H](C)C(=O)O"
ibp=Chem.MolFromSmiles(ibp_s)

display(ibp)
ibpH = Chem.AddHs(ibp)
AllChem.EmbedMolecule(ibpH, AllChem.ETKDGv3())
display(ibpH)

In [ ]:
Descriptors.NumRotatableBonds(ibp)

In [ ]:
cel=Chem.MolFromSmiles("Cc1ccc(cc1)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)(F)F")
cel

In [ ]:
Descriptors.NumRotatableBonds(cel)

# Prepare a "library".

I.e., a single SDF files with the 3 candidates. This will be our "library". Actual virtual screening efforts use libraries of thousands to millions of compounds. Of course, they are parallelized over CPUs and nodes.

Note that the following are not granted:
 * Drug-likeness
 * Purchase-ability
 * Synthetic accessibility
 * Stereochemistry
 * Stability
 * ...and so on.

 Since we are here, add a couple of properties that are carried together with the molecule. (InchiKey is the hashed version of the standard "International Chemical Identifier (InChI)")

In [ ]:
with Chem.SDWriter('mylibrary.sdf') as w:
  for m in [ain, ibp, cel]:
    m.SetProp("SMILES", Chem.MolToSmiles(m))
    m.SetProp("InchiKey", Chem.MolToInchiKey(m))
    w.write(m)


In [ ]:
%cat mylibrary.sdf

Protonate the library for pH 7 and generate 3D conformers. (We could have done the 3D conformer generation with RDKit as well).

In [ ]:
!obabel mylibrary.sdf -O mylibrary_p.sdf --gen3D -p7

In [ ]:
%cat mylibrary_p.sdf

# Docking

Now we do some pre-preparation on the receptor. Dock a few NSAIDs to the cyclooxygenase 2 (COX-2) target.

We'll use these structures:
 * 3LN1: "Structure of celecoxib bound at the COX-2 active site". (Mouse)
 * 1EQG: "THE 2.6 ANGSTROM MODEL OF OVINE COX-1 COMPLEXED WITH IBUPROFEN"

We'll dock to the former, and validate on the latter.

In [ ]:
import mdtraj as mdt
cox2 = mdt.load_pdb("https://files.rcsb.org/download/3LN1.pdb")

In [ ]:
# Extract biological assembly (dimer), protein only
cox2ba = cox2.atom_slice(
    cox2.topology.select("protein and chainid 0 or chainid 1")
)
cox2ba.save("rec.pdb")

In [ ]:
# ONLY one ligand, used for defining the binding site
lig_4 = cox2.atom_slice(
    cox2.topology.select("resname CEL")
)
lig_1 = lig_4.atom_slice(
    lig_4.topology.select("chainid 0")
)
lig_1.save("lig.pdb")

In [ ]:
!smina -r rec.pdb -l mylibrary_p.sdf --autobox_ligand lig.pdb -o smina_out.sdf

# How did it go?

Now load the SDF results (note that they are sorted by decreasing score) together with the target.

Compare e.g. with the experimental pose of aspirin in 1EQG (ovine). Make sure to align the proteins.



# Now with flexible docking

In [ ]:
!smina --help

In [ ]:
# Too slow to run during the class
# !smina -r rec.pdb -l mylibrary_p.sdf --autobox_ligand lig.pdb --flexdist 4 --flexdist_ligand lig.pdb -o smina_out_flex.sdf

# Random notes

Some tutorials insist on the pdbqt format. However this seems to be immaterial. To start, smina does not read charges from files.

```
!obabel rec.pdb -p7 -xr -O rec.pdbqt
!obabel lig.pdb -p7 -O lig.pdbqt
!obabel mylibrary_p.sdf -p7 -O mylibrary_p.pdbqt

!smina -r rec.pdbqt -l mylibrary_p.sdf --autobox_ligand lig.pdb -o smina_out_q.sdf
```




In [ ]:
!obabel '-:CC(C)Cc1ccc(cc1)[C@H](C)C(=O)O' --gen3D -p7 -O temp.pdbqt --partialcharge eem
!cat temp.pdbqt

In [ ]:
!obabel -L charges

In [ ]:
# Unused. Emergency way to filter stuff
"""
!grep -v HETATM 3LN1.pdb > rec.pdb
!grep "CEL A" 3LN1.pdb > lig.pdb
"""

In [ ]:
# Openbabel can also add properties
!obabel mylibrary.sdf -O test.sdf --gen3D -p7  --add cansmi
!cat test.sdf

In [ ]:
!obabel -L descriptors